In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("twitter_human_bots_dataset_corrupted.csv")

print("Shape:", df.shape)
print("\nColumns and types:")
print(df.dtypes)
print("\nFirst few rows:")
print(df.head())
print("\nBasic stats:")
print(df.describe())
print("\nMissing values:")
print(df.isnull().sum())

print(df['account_type'].value_counts())

C:\Users\Badr\AppData\Local\Temp\ipykernel_15956\3373649270.py:1: DtypeWarning: Columns (0,3,5,6,7,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("twitter_human_bots_dataset_corrupted.csv")


Shape: (37446, 20)

Columns and types:
Unnamed: 0                      object
created_at                      object
default_profile                 object
default_profile_image           object
description                     object
favourites_count                object
followers_count                 object
friends_count                   object
geo_enabled                     object
id                              object
lang                            object
location                        object
profile_background_image_url    object
profile_image_url               object
screen_name                     object
statuses_count                  object
verified                        object
average_tweets_per_day          object
account_age_days                object
account_type                    object
dtype: object

First few rows:
  Unnamed: 0           created_at default_profile default_profile_image  \
0          0  2016-10-15 21:32:11           False                 False   


In [ ]:
# cleaning label
print("\nValue Counts:")
print(df['account_type'].value_counts())

print(f"Missing values: {df['account_type'].isna().sum()}")
print(f"Total rows: {len(df)}")

# Step 1: Standardize to lowercase
df['account_type'] = df['account_type'].str.lower().str.strip()

# Step 2: Map variations to standard values
mapping = {
    'human': 'human',
    'bot': 'bot',
    'invalid_bot': 'bot',  
    '7jqfi': np.nan,       
    'namuh': 'human',      
    'tew': np.nan          
}

df['account_type'] = df['account_type'].map(mapping)

# drop nan
print("\n" + "="*50)
print("CLEANING ACTIONS:")
print("="*50)
print("- Standardized all values to lowercase")
print("- Mapped 'invalid_bot' -> 'bot'")
print("- Mapped 'namuh' (backwards) -> 'human'")
print("- Set clearly invalid values ('7jqfi', 'tew') to NaN")
print("- Dropping all rows with NaN in account_type")

df_clean = df.dropna(subset=['account_type'])

print("\n" + "="*50)
print("AFTER CLEANING:")
print("="*50)
print(f"Total rows: {len(df_clean)}")
print(f"Rows dropped: {len(df) - len(df_clean)}")
print(f"\nValue counts:\n{df_clean['account_type'].value_counts()}")
print(f"Missing values: {df_clean['account_type'].isna().sum()}")



Value Counts:
account_type
human          25002
bot            12418
INVALID_bot        2
HUMAN              2
7JQFi              1
namuh              1
teW                1
Name: count, dtype: int64
Missing values: 19
Total rows: 37446

CLEANING ACTIONS:
- Standardized all values to lowercase
- Mapped 'invalid_bot' -> 'bot'
- Mapped 'namuh' (backwards) -> 'human'
- Set clearly invalid values ('7jqfi', 'tew') to NaN
- Dropping all rows with NaN in account_type

AFTER CLEANING:
Total rows: 37425
Rows dropped: 21

Value counts:
account_type
human    25005
bot      12420
Name: count, dtype: int64
Missing values: 0


In [4]:
# Examine Boolean Features

print("="*70)
print("EXAMINING BOOLEAN FEATURES")
print("="*70)
print(f"Total rows: {len(df)}\n")

# Boolean columns to examine
boolean_cols = ['default_profile', 'default_profile_image', 'verified', 'geo_enabled']

for col in boolean_cols:
    print("="*70)
    print(f"COLUMN: {col}")
    print("="*70)
    
    # Show data type
    print(f"Data type: {df[col].dtype}")
    
    # Show unique values
    print(f"\nUnique values: {df[col].nunique()}")
    print("\nValue counts:")
    print(df[col].value_counts(dropna=False))
    
    # Show percentage
    print("\nPercentage distribution:")
    print(df[col].value_counts(normalize=True, dropna=False) * 100)
    
    # Check for missing values
    missing = df[col].isnull().sum()
    print(f"\nMissing values: {missing} ({missing/len(df)*100:.2f}%)")
    
    # Check for non-boolean values (if stored as object/string)
    if df[col].dtype == 'object':
        print("\n⚠️  WARNING: Column is stored as object/string, not boolean")
        print("Sample values:", df[col].head(10).tolist())
    
    print("\n")

# Summary
print("="*70)
print("SUMMARY & RECOMMENDATIONS")
print("="*70)
print("\nNext steps:")
print("1. Check if values need to be converted to proper boolean type")
print("2. Decide how to handle missing values")
print("3. Look for any unexpected values (e.g., 'True' as string vs True as bool)")

EXAMINING BOOLEAN FEATURES
Total rows: 37446

COLUMN: default_profile
Data type: object

Unique values: 14

Value counts:
default_profile
False      21704
True       15706
NaN           13
eslaF          7
eurT           5
unknown        2
0              1
-9999          1
161            1
FeNf5          1
3706           1
FALSE          1
TRUE           1
676            1
B01X           1
Name: count, dtype: int64

Percentage distribution:
default_profile
False      57.960797
True       41.943065
NaN         0.034717
eslaF       0.018694
eurT        0.013353
unknown     0.005341
0           0.002671
-9999       0.002671
161         0.002671
FeNf5       0.002671
3706        0.002671
FALSE       0.002671
TRUE        0.002671
676         0.002671
B01X        0.002671
Name: proportion, dtype: float64

Missing values: 13 (0.03%)

⚠️  WARNING: Column is stored as object/string, not boolean
Sample values: ['False', 'False', 'False', 'True', 'False', 'True', 'False', 'False', 'True', 'False']

In [ ]:
#cleaning default_profiile

print("="*70)
print("CLEANING: default_profile")
print("="*70)

print("\nBEFORE CLEANING:")
print(f"Data type: {df['default_profile'].dtype}")
print(f"Unique values: {df['default_profile'].nunique()}")
print("\nValue counts:")
print(df['default_profile'].value_counts(dropna=False))

# lowercase
df['default_profile'] = df['default_profile'].astype(str).str.lower().str.strip()

# map
def map_to_boolean(value):
    # if true
    if value in ['true', '1', 'yes', 'eurt']:
        return True
    #if flase
    elif value in ['false', '0', 'no', 'eslaf']:  # 'eslaf' is 'false' backwards
        return False
    # missing
    elif value in ['nan', 'none', 'unknown', '-9999']:
        return np.nan
    #else = nan
    else:
        return np.nan

df['default_profile'] = df['default_profile'].apply(map_to_boolean)

print("\n" + "="*70)
print("AFTER CLEANING:")
print("="*70)
print(f"Data type: {df['default_profile'].dtype}")
print(f"Unique values: {df['default_profile'].nunique()}")
print("\nValue counts:")
print(df['default_profile'].value_counts(dropna=False))

# Calculate how many were converted to NaN
missing_after = df['default_profile'].isnull().sum()
print(f"\nTotal missing/invalid values: {missing_after} ({missing_after/len(df)*100:.2f}%)")

# Show what was mapped to NaN (for verification)
print("\n" + "="*70)
print("CLEANING SUMMARY:")
print("="*70)
print(" Standardized 'True'/'TRUE'/'true' → True")
print(" Standardized 'False'/'FALSE'/'false' → False")
print("Mapped 'eurT' (backwards) → True")
print(" Mapped 'eslaF' (backwards) → False")
print("Set invalid values (numbers, random strings) → NaN")

print(f"\nRows with True: {(df['default_profile'] == True).sum()}")
print(f"Rows with False: {(df['default_profile'] == False).sum()}")
print(f"Rows with NaN: {df['default_profile'].isna().sum()}")

# impute missing values
print("\n" + "="*70)
print("HANDLING MISSING VALUES:")
print("="*70)
print(f"Missing values before imputation: {df['default_profile'].isna().sum()}")

# Get the most common value (mode)
mode_value = df['default_profile'].mode()[0]
print(f"Most common value (mode): {mode_value}")

# Impute NaN with mode
df['default_profile'] = df['default_profile'].fillna(mode_value)

print(f"Missing values after imputation: {df['default_profile'].isna().sum()}")
print("All missing values imputed with False")

# Final value counts
print("\n" + "="*70)
print("FINAL VALUE COUNTS:")
print("="*70)
print(df['default_profile'].value_counts(dropna=False))
print(f"\nTrue: {(df['default_profile'] == True).sum()} ({(df['default_profile'] == True).sum()/len(df)*100:.2f}%)")
print(f"False: {(df['default_profile'] == False).sum()} ({(df['default_profile'] == False).sum()/len(df)*100:.2f}%)")

CLEANING: default_profile

BEFORE CLEANING:
Data type: object
Unique values: 14

Value counts:
default_profile
False      21704
True       15706
NaN           13
eslaF          7
eurT           5
unknown        2
0              1
-9999          1
161            1
FeNf5          1
3706           1
FALSE          1
TRUE           1
676            1
B01X           1
Name: count, dtype: int64

AFTER CLEANING:
Data type: object
Unique values: 2

Value counts:
default_profile
False    21713
True     15712
NaN         21
Name: count, dtype: int64

Total missing/invalid values: 21 (0.06%)

CLEANING SUMMARY:
✓ Standardized 'True'/'TRUE'/'true' → True
✓ Standardized 'False'/'FALSE'/'false' → False
✓ Mapped 'eurT' (backwards) → True
✓ Mapped 'eslaF' (backwards) → False
✓ Set invalid values (numbers, random strings) → NaN

Rows with True: 15712
Rows with False: 21713
Rows with NaN: 21

HANDLING MISSING VALUES:
Missing values before imputation: 21
Most common value (mode): False
Missing values afte

C:\Users\Badr\AppData\Local\Temp\ipykernel_15956\2120525870.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['default_profile'] = df['default_profile'].fillna(mode_value)


In [ ]:
# Clean the default_profile_image column

print("="*70)
print("CLEANING: default_profile_image")
print("="*70)

print("\nBEFORE CLEANING:")
print(f"Data type: {df['default_profile_image'].dtype}")
print(f"Unique values: {df['default_profile_image'].nunique()}")
print("\nValue counts:")
print(df['default_profile_image'].value_counts(dropna=False))

# lowercase
df['default_profile_image'] = df['default_profile_image'].astype(str).str.lower().str.strip()

# mapping
def map_to_boolean(value):
    #  True 
    if value in ['true', '1', 'yes', 'eurt']:  
        return True
    #  False 
    elif value in ['false', '0', 'no', 'eslaf', 'invalid_false']:  
        return False
    #  missing/invalid 
    elif value in ['nan', 'none', 'unknown', '-9999', '999999']:
        return np.nan
    else:
        return np.nan

df['default_profile_image'] = df['default_profile_image'].apply(map_to_boolean)

print("\n" + "="*70)
print("AFTER CLEANING:")
print("="*70)
print(f"Data type: {df['default_profile_image'].dtype}")
print(f"Unique values: {df['default_profile_image'].nunique()}")
print("\nValue counts:")
print(df['default_profile_image'].value_counts(dropna=False))

# nan number
missing_after = df['default_profile_image'].isnull().sum()
print(f"\nTotal missing/invalid values: {missing_after} ({missing_after/len(df)*100:.2f}%)")

# imputing
print("\n" + "="*70)
print("HANDLING MISSING VALUES:")
print("="*70)
print(f"Missing values before imputation: {df['default_profile_image'].isna().sum()}")

# Get the most common value (mode)
mode_value = df['default_profile_image'].mode()[0]
print(f"Most common value (mode): {mode_value}")

# Impute NaN with mode
df['default_profile_image'] = df['default_profile_image'].fillna(mode_value)

print(f"Missing values after imputation: {df['default_profile_image'].isna().sum()}")
print(f"✓ All missing values imputed with {mode_value}")

# Final value counts
print("\n" + "="*70)
print("FINAL VALUE COUNTS:")
print("="*70)
print(df['default_profile_image'].value_counts(dropna=False))
print(f"\nTrue: {(df['default_profile_image'] == True).sum()} ({(df['default_profile_image'] == True).sum()/len(df)*100:.2f}%)")
print(f"False: {(df['default_profile_image'] == False).sum()} ({(df['default_profile_image'] == False).sum()/len(df)*100:.2f}%)")



CLEANING: default_profile_image

BEFORE CLEANING:
Data type: object
Unique values: 14

Value counts:
default_profile_image
False            32243
False             4615
True               495
True                63
NaN                  9
FALSE                4
0                    4
eslaF                4
-9999                2
INVALID_False        2
JuaP6                1
GHOdo                1
unknown              1
999999               1
lsUB6                1
Name: count, dtype: int64

AFTER CLEANING:
Data type: object
Unique values: 2

Value counts:
default_profile_image
False    36872
True       558
NaN         16
Name: count, dtype: int64

Total missing/invalid values: 16 (0.04%)

HANDLING MISSING VALUES:
Missing values before imputation: 16
Most common value (mode): False
Missing values after imputation: 0
✓ All missing values imputed with False

FINAL VALUE COUNTS:
default_profile_image
False    36888
True       558
Name: count, dtype: int64

True: 558 (1.49%)
False: 36888 (98

C:\Users\Badr\AppData\Local\Temp\ipykernel_15956\3490893910.py:56: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['default_profile_image'] = df['default_profile_image'].fillna(mode_value)


In [ ]:

# Clean the verified column
print("="*70)
print("CLEANING: verified")
print("="*70)

print("\nBEFORE CLEANING:")
print(f"Data type: {df['verified'].dtype}")
print(f"Unique values: {df['verified'].nunique()}")
print("\nValue counts:")
print(df['verified'].value_counts(dropna=False))

# lowercase
df['verified'] = df['verified'].astype(str).str.lower().str.strip()

# mapping
def map_to_boolean(value):
    if value in ['true', '1', 'yes', 'eurt', 'invalid_true', 'eurt']:
        return True
   
    elif value in ['false', '0', 'no', 'eslaf', 'invalid_false']:
        return False
    
    elif value in ['nan', 'none', 'unknown', '-9999', '999999', '61q3s', 'otnx']:
        return np.nan
    else:
        return np.nan

df['verified'] = df['verified'].apply(map_to_boolean)

print("\n" + "="*70)
print("AFTER MAPPING:")
print("="*70)
print(f"Data type: {df['verified'].dtype}")
print(f"Unique values: {df['verified'].nunique()}")
print("\nValue counts:")
print(df['verified'].value_counts(dropna=False))

# Calculate how many were converted to NaN (the total count of 'np.nan')
missing_after = df['verified'].isnull().sum()
print(f"\nTotal missing/invalid values: {missing_after} ({missing_after/len(df)*100:.2f}%)")

# impute missing values
print("\n" + "="*70)
print("HANDLING MISSING VALUES:")
print("="*70)
print(f"Missing values before imputation: {df['verified'].isna().sum()}")

mode_value = df['verified'].mode()[0]
print(f"Most common value (mode): {mode_value}")

df['verified'] = df['verified'].fillna(mode_value)
df['verified'] = df['verified'].astype(bool)

print(f"Missing values after imputation: {df['verified'].isna().sum()}")
print(f"✓ All missing values imputed with {mode_value}")

print("\n" + "="*70)
print("FINAL VALUE COUNTS:")
print("="*70)
print(df['verified'].value_counts(dropna=False))
print(f"\nTrue: {(df['verified'] == True).sum()} ({(df['verified'] == True).sum()/len(df)*100:.2f}%)")
print(f"False: {(df['verified'] == False).sum()} ({(df['verified'] == False).sum()/len(df)*100:.2f}%)")

# Show what was cleaned
print("\n" + "="*70)
print("CLEANING SUMMARY:")
print("="*70)
print("✓ Standardized 'True'/'FALSE'/'false' → True/False")
print("✓ Mapped 'eslaF' (backwards) and 'INVALID_False' → False")
print("✓ Mapped 'eurT' and 'INVALID_True' → True")
print("✓ Set invalid values (numbers, random strings) → NaN (Total: 34 values)")
print(f"✓ Imputed all NaN with mode ({mode_value})")
print(f"✓ Final Data Type: {df['verified'].dtype}")

CLEANING: verified

BEFORE CLEANING:
Data type: object
Unique values: 12

Value counts:
verified
False            29861
True              7545
NaN                 19
FALSE                5
eslaF                4
INVALID_False        3
INVALID_True         2
eurT                 2
999999               1
61q3S                1
OtnX                 1
-9999                1
unknown              1
Name: count, dtype: int64

AFTER MAPPING:
Data type: object
Unique values: 2

Value counts:
verified
False    29873
True      7549
NaN         24
Name: count, dtype: int64

Total missing/invalid values: 24 (0.06%)

HANDLING MISSING VALUES:
Missing values before imputation: 24
Most common value (mode): False
Missing values after imputation: 0
✓ All missing values imputed with False

FINAL VALUE COUNTS:
verified
False    29897
True      7549
Name: count, dtype: int64

True: 7549 (20.16%)
False: 29897 (79.84%)

CLEANING SUMMARY:
✓ Standardized 'True'/'FALSE'/'false' → True/False
✓ Mapped 'eslaF' (bac

C:\Users\Badr\AppData\Local\Temp\ipykernel_15956\3464289228.py:64: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['verified'] = df['verified'].fillna(mode_value)


In [ ]:

# Clean the geo_enabled column

print("="*70)
print("CLEANING: geo_enabled")
print("="*70)

print("\nBEFORE CLEANING:")
print(f"Data type: {df['geo_enabled'].dtype}")
print(f"Unique values: {df['geo_enabled'].nunique()}")
print("\nValue counts:")
print(df['geo_enabled'].value_counts(dropna=False))

# lowercase
df['geo_enabled'] = df['geo_enabled'].astype(str).str.lower().str.strip()

# Step 2: Map all variations to proper boolean values
def map_to_boolean_geo(value):
   
    if value in ['true', '1', 'yes', 'eurt', 'invalid_true']:
        return True
    
    elif value in ['false', '0', 'no', 'eslaf', 'invalid_false']:
        return False
    
    elif value in ['nan', 'none', 'unknown', '-9999', '999999', 
                    'http://abs.twimg.com/images/themes/theme1/bg.png', 
                    'http://abs.twimg.com/images/themes/theme10/bg.gif', 
                    'qhaq', 'lh1f', 'knr4']:
        return np.nan
    
    else:
        return np.nan

df['geo_enabled'] = df['geo_enabled'].apply(map_to_boolean_geo)

print("\n" + "="*70)
print("AFTER MAPPING:")
print("="*70)
print(f"Data type: {df['geo_enabled'].dtype}")
print(f"Unique values: {df['geo_enabled'].nunique()}")
print("\nValue counts:")
print(df['geo_enabled'].value_counts(dropna=False))

missing_after = df['geo_enabled'].isnull().sum()
print(f"\nTotal missing/invalid values: {missing_after} ({missing_after/len(df)*100:.2f}%)")


#imputing
print("\n" + "="*70)
print("HANDLING MISSING VALUES:")
print("="*70)
print(f"Missing values before imputation: {df['geo_enabled'].isna().sum()}")
mode_value = df['geo_enabled'].mode()[0]
print(f"Most common value (mode): {mode_value}")

df['geo_enabled'] = df['geo_enabled'].fillna(mode_value)

df['geo_enabled'] = df['geo_enabled'].astype(bool)

print(f"Missing values after imputation: {df['geo_enabled'].isna().sum()}")
print(f"All missing values imputed with {mode_value}")

# Final value counts
print("\n" + "="*70)
print("FINAL VALUE COUNTS:")
print("="*70)
print(df['geo_enabled'].value_counts(dropna=False))
print(f"\nTrue: {(df['geo_enabled'] == True).sum()} ({(df['geo_enabled'] == True).sum()/len(df)*100:.2f}%)")
print(f"False: {(df['geo_enabled'] == False).sum()} ({(df['geo_enabled'] == False).sum()/len(df)*100:.2f}%)")


CLEANING: geo_enabled

BEFORE CLEANING:
Data type: object
Unique values: 16

Value counts:
geo_enabled
False                                                20343
True                                                 17058
NaN                                                     13
eslaF                                                    7
TRUE                                                     4
eurT                                                     3
unknown                                                  3
999999                                                   3
FALSE                                                    2
-9999                                                    2
http://abs.twimg.com/images/themes/theme1/bg.png         2
http://abs.twimg.com/images/themes/theme10/bg.gif        1
QHAQ                                                     1
INVALID_False                                            1
INVALID_True                                             1
Lh1F        

C:\Users\Badr\AppData\Local\Temp\ipykernel_15956\3447776333.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['geo_enabled'] = df['geo_enabled'].fillna(mode_value)


In [ ]:
# List of numeric columns that are currently objects
numeric_cols_to_inspect = [
    'favourites_count', 
    'followers_count', 
    'friends_count', 
    'statuses_count'
]

print("="*70)
print("INSPECTING NON-STANDARD VALUES IN NUMERIC COLUMNS")
print("="*70)
print("Finding all unique non-numeric strings or invalid placeholders:")

# Dictionary to store unique garbage values for each column
garbage_values = {}
total_garbage_count = 0

for col in numeric_cols_to_inspect:
    # Get all unique values for the column
    unique_values = df[col].astype(str).str.strip().unique()
    
    # Attempt to convert each unique value to numeric. If it fails (is an error), it's garbage.
    # We also check for the specific numeric placeholder '-9999'.
    garbage_list = []
    
    for val in unique_values:
        # Check if the value is NaN or if it successfully converts to a number
        try:
            # Check for the specific numeric placeholder
            if float(val) == -9999.0:
                garbage_list.append(val)
            
            # Check for other garbage values (like random strings)
            # If conversion fails, the code jumps to the 'except ValueError' block below.
            float(val)
            
        except ValueError:
            # This block captures non-numeric strings like 'unknown', 'eslaF', etc.
            if val.lower() != 'nan': # Exclude the string 'nan' from the report if it came from NaNs
                garbage_list.append(val)

    if garbage_list:
        # Print only the garbage found in this column
        print(f"\n--- {col} ({len(garbage_list)} unique garbage/placeholder values) ---")
        print(set(garbage_list))
        
        # Add to the running total (using a set to ensure unique values across all columns)
        for g_val in garbage_list:
            if g_val not in garbage_values:
                garbage_values[g_val] = 0
            garbage_values[g_val] += 1
            total_garbage_count += 1

if not garbage_values:
    print("\nNo further unique non-numeric garbage strings found beyond what 'pd.to_numeric(errors='coerce')' handles!")

# Print the final set of all unique garbage values found across all columns
print("\n" + "="*70)
print(f"SUMMARY: All Unique Identified Garbage Values ({len(garbage_values)} types):")
print("="*70)
print(set(garbage_values.keys()))

🔍 INSPECTING NON-STANDARD VALUES IN NUMERIC COLUMNS
Finding all unique non-numeric strings or invalid placeholders:

--- favourites_count (4 unique garbage/placeholder values) ---
{'dV', 'INVALID_1754', 'fVR', 'unknown'}

--- followers_count (10 unique garbage/placeholder values) ---
{'INVALID_719', 'gKw', 'unknown', '-9999', 'INVALID_10', '-9999.0', 'en', 'INVALID_913', 'nI1naKB', 'ar'}

--- friends_count (12 unique garbage/placeholder values) ---
{'5Ej', 'where destination be.', '47.41155,8.543795', 'unknown', '-9999', 'INVALID_507', 'INVALID_845', 'INVALID_2115', 'INVALID_425', 'INVALID_0', 'INVALID_337', 'New York'}

--- statuses_count (7 unique garbage/placeholder values) ---
{'human', 'unknown', '-9999', 'INVALID_308', 'INVALID_1843', 'rhEVa2', 'bot'}

SUMMARY: All Unique Identified Garbage Values (28 types):
{'gKw', 'unknown', '47.41155,8.543795', 'INVALID_10', 'INVALID_0', 'dV', 'INVALID_719', 'INVALID_1754', 'INVALID_2115', 'en', 'fVR', 'INVALID_913', 'INVALID_337', 'ar', 'INV

In [ ]:
# CLEANING: friends_count 

print("="*70)
print("CLEANING: friends_count")
print("="*70)

# convert to numeric, all non numeric values ==> nan
df['friends_count'] = pd.to_numeric(df['friends_count'], errors='coerce')


# df.loc[df['friends_count'] <= -9999, 'friends_count'] = np.nan
print("\n" + "="*70)
print("AFTER COERCION AND GARBAGE FIX:")
print("="*70)
print(f"Data type: {df['friends_count'].dtype}")
missing_after_fix = df['friends_count'].isnull().sum()
print(f"Total missing/invalid values (NaNs): {missing_after_fix}")

# calculate median
print("\n" + "="*70)
print("RECALCULATING MEDIAN FOR IMPUTATION:")
print("="*70)
median_value = df['friends_count'].median()
print(f"New Median (calculated from clean data): {median_value:.0f}")

# Impute missing with median
print("\n" + "="*70)
print("IMPUTING MISSING VALUES:")
print("="*70)
print(f"Missing values before imputation: {missing_after_fix}")
df['friends_count'] = df['friends_count'].fillna(median_value)

print(f"Missing values after imputation: {df['friends_count'].isna().sum()}")
print(f"✓ All missing values imputed with {median_value:.0f}")


# convert to int
df['friends_count'] = df['friends_count'].astype('int64')

print("\n" + "="*70)
print("FINAL CLEANING SUMMARY:")
print("="*70)
print(f"Final Data Type: {df['friends_count'].dtype}")
print(f"Final Missing Values: {df['friends_count'].isnull().sum()}")
print("\n✓ 'friends_count' column successfully cleaned and converted to 'int64'")

CLEANING: friends_count

AFTER COERCION AND GARBAGE FIX:
Data type: float64
Total missing/invalid values (NaNs): 28

RECALCULATING MEDIAN FOR IMPUTATION:
New Median (calculated from clean data): 296

IMPUTING MISSING VALUES:
Missing values before imputation: 28
Missing values after imputation: 0
✓ All missing values imputed with 296

FINAL CLEANING SUMMARY:
Final Data Type: int64
Final Missing Values: 0

✓ 'friends_count' column successfully cleaned and converted to 'int64'


In [ ]:

# CLEANING: favourites_count 

print("="*70)
print("CLEANING: favourites_count")
print("="*70)

# convert to numeric, all non numeric values ==> nan
df['favourites_count'] = pd.to_numeric(df['favourites_count'], errors='coerce')

#df.loc[df['favourites_count'] <= -9999, 'favourites_count'] = np.nan

print("\n" + "="*70)
print("AFTER COERCION AND GARBAGE FIX:")
print("="*70)
print(f"Data type: {df['favourites_count'].dtype}")
missing_after_fix = df['favourites_count'].isnull().sum()
print(f"Total missing/invalid values (NaNs): {missing_after_fix}")

# meidon claculation
print("\n" + "="*70)
print("RECALCULATING MEDIAN FOR IMPUTATION:")
print("="*70)
median_value = df['favourites_count'].median()
print(f"New Median (calculated from clean data): {median_value:.0f}")

# impute missing with median
print("\n" + "="*70)
print("IMPUTING MISSING VALUES:")
print("="*70)
print(f"Missing values before imputation: {missing_after_fix}")

df['favourites_count'] = df['favourites_count'].fillna(median_value)

print(f"Missing values after imputation: {df['favourites_count'].isna().sum()}")
print(f"All missing values imputed with {median_value:.0f}")


#convert to int
df['favourites_count'] = df['favourites_count'].astype('int64')

# --- Final Inspection ---
print("\n" + "="*70)
print("FINAL CLEANING SUMMARY:")
print("="*70)
print(f"Final Data Type: {df['favourites_count'].dtype}")
print(f"Final Missing Values: {df['favourites_count'].isnull().sum()}")
print("\n✓ 'favourites_count' column successfully cleaned and converted to 'int64'")

CLEANING: favourites_count

AFTER COERCION AND GARBAGE FIX:
Data type: float64
Total missing/invalid values (NaNs): 18

RECALCULATING MEDIAN FOR IMPUTATION:
New Median (calculated from clean data): 2068

IMPUTING MISSING VALUES:
Missing values before imputation: 18
Missing values after imputation: 0
✓ All missing values imputed with 2068

FINAL CLEANING SUMMARY:
Final Data Type: int64
Final Missing Values: 0

✓ 'favourites_count' column successfully cleaned and converted to 'int64'


In [ ]:
# CLEANING: statuses_count

print("="*70)
print("CLEANING: statuses_count")
print("="*70)

# convert to numeric, all non numeric values ==> nan
df['statuses_count'] = pd.to_numeric(df['statuses_count'], errors='coerce')

# Step 2: Explicitly map the recognized numeric placeholders to NaN
# We map any value that is -9999 or lower to catch the placeholder.
df.loc[df['statuses_count'] <= -9999, 'statuses_count'] = np.nan

print("\n" + "="*70)
print("AFTER COERCION AND GARBAGE FIX:")
print("="*70)
print(f"Data type: {df['statuses_count'].dtype}")
missing_after_fix = df['statuses_count'].isnull().sum()
print(f"Total missing/invalid values (NaNs): {missing_after_fix}")

# median calculation
print("\n" + "="*70)
print("RECALCULATING MEDIAN FOR IMPUTATION:")
print("="*70)
median_value = df['statuses_count'].median()
print(f"New Median (calculated from clean data): {median_value:.0f}")

# impute with median
print("\n" + "="*70)
print("IMPUTING MISSING VALUES:")
print("="*70)
print(f"Missing values before imputation: {missing_after_fix}")
df['statuses_count'] = df['statuses_count'].fillna(median_value)

print(f"Missing values after imputation: {df['statuses_count'].isna().sum()}")
print(f" All missing values imputed with {median_value:.0f}")


#convert to int
df['statuses_count'] = df['statuses_count'].astype('int64')

# --- Final Inspection ---
print("\n" + "="*70)
print("FINAL CLEANING SUMMARY:")
print("="*70)
print(f"Final Data Type: {df['statuses_count'].dtype}")
print(f"Final Missing Values: {df['statuses_count'].isnull().sum()}")
print("\n✓ 'statuses_count' column successfully cleaned and converted to 'int64'")

CLEANING: statuses_count

AFTER COERCION AND GARBAGE FIX:
Data type: float64
Total missing/invalid values (NaNs): 26

RECALCULATING MEDIAN FOR IMPUTATION:
New Median (calculated from clean data): 4210

IMPUTING MISSING VALUES:
Missing values before imputation: 26
Missing values after imputation: 0
✓ All missing values imputed with 4210

FINAL CLEANING SUMMARY:
Final Data Type: int64
Final Missing Values: 0

✓ 'statuses_count' column successfully cleaned and converted to 'int64'


In [ ]:
# CLEANING: followers_count 

print("="*70)
print("CLEANING: followers_count")
print("="*70)

# convert to numeric, all non numeric values ==> nan

df['followers_count'] = pd.to_numeric(df['followers_count'], errors='coerce')

# map -9999 to nan
df.loc[df['followers_count'] <= -9999, 'followers_count'] = np.nan

print("\n" + "="*70)
print("AFTER COERCION AND GARBAGE FIX:")
print("="*70)
print(f"Data type: {df['followers_count'].dtype}")
missing_after_fix = df['followers_count'].isnull().sum()
print(f"Total missing/invalid values (NaNs): {missing_after_fix}")

# calculate median
print("\n" + "="*70)
print("RECALCULATING MEDIAN FOR IMPUTATION:")
print("="*70)
median_value = df['followers_count'].median()
print(f"New Median (calculated from clean data): {median_value:.0f}")

# impute
print("\n" + "="*70)
print("IMPUTING MISSING VALUES:")
print("="*70)
print(f"Missing values before imputation: {missing_after_fix}")
df['followers_count'] = df['followers_count'].fillna(median_value)

print(f"Missing values after imputation: {df['followers_count'].isna().sum()}")
print(f"✓ All missing values imputed with {median_value:.0f}")


#convert tot int
df['followers_count'] = df['followers_count'].astype('int64')

print("\n" + "="*70)
print("FINAL CLEANING SUMMARY:")
print("="*70)
print(f"Final Data Type: {df['followers_count'].dtype}")
print(f"Final Missing Values: {df['followers_count'].isnull().sum()}")
print("\n✓ 'followers_count' column successfully cleaned and converted to 'int64'")

CLEANING: followers_count

AFTER COERCION AND GARBAGE FIX:
Data type: float64
Total missing/invalid values (NaNs): 25

RECALCULATING MEDIAN FOR IMPUTATION:
New Median (calculated from clean data): 365

IMPUTING MISSING VALUES:
Missing values before imputation: 25
Missing values after imputation: 0
✓ All missing values imputed with 365

FINAL CLEANING SUMMARY:
Final Data Type: int64
Final Missing Values: 0

✓ 'followers_count' column successfully cleaned and converted to 'int64'


In [ ]:

# List of columns to inspect
time_metrics_to_inspect = [
    'average_tweets_per_day', 
    'account_age_days'
]

print("="*70)
print("🔍 INSPECTING NON-STANDARD VALUES IN TIME-BASED METRICS")
print("="*70)
print("Finding all unique non-numeric strings or invalid placeholders:")

garbage_values = {}

for col in time_metrics_to_inspect:
    unique_values = df[col].astype(str).str.strip().unique()
    
    garbage_list = []
    
    for val in unique_values:
        if val.startswith('-9999'):
            garbage_list.append(val)
            continue
        
        try:
            # Check if the value is NaN
            if val.lower() == 'nan':
                continue
            
            # If conversion fails, the code jumps to the 'except ValueError' block below.
            float(val)
            
        except ValueError:
            # This block captures non-numeric strings like 'unknown', 'bot', etc.
            garbage_list.append(val)

    if garbage_list:
        # Print only the garbage found in this column
        print(f"\n--- {col} ({len(garbage_list)} unique garbage/placeholder values) ---")
        print(set(garbage_list))
        
        # Add to the running set of all garbage found
        for g_val in garbage_list:
            garbage_values[g_val] = garbage_values.get(g_val, 0) + 1

# Print the final set of all unique garbage values found across all columns
print("\n" + "="*70)
print(f"SUMMARY: All Unique Identified Garbage Values ({len(garbage_values)} types):")
print("="*70)
print(set(garbage_values.keys()))

🔍 INSPECTING NON-STANDARD VALUES IN TIME-BASED METRICS
Finding all unique non-numeric strings or invalid placeholders:

--- average_tweets_per_day (5 unique garbage/placeholder values) ---
{'unknown', 'INVALID_1.961', 'DujwPd', 'INVALID_1.021', 'INVALID_11.035'}

--- account_age_days (9 unique garbage/placeholder values) ---
{'FbTT', 'sMFE', 'XGp0', '-9999', '53m8', 'vIIz', 'INVALID_3033', 'INVALID_3490', 'i9ve'}

SUMMARY: All Unique Identified Garbage Values (14 types):
{'FbTT', 'sMFE', 'XGp0', 'unknown', '-9999', '53m8', 'INVALID_1.961', 'DujwPd', 'vIIz', 'INVALID_3033', 'INVALID_1.021', 'INVALID_11.035', 'INVALID_3490', 'i9ve'}


In [ ]:

# CLEANING: average_tweets_per_day

print("="*70)
print("CLEANING: average_tweets_per_day")
print("="*70)

# convert to string
df['average_tweets_per_day'] = df['average_tweets_per_day'].astype(str).str.strip()

# dealing with INVALID_ prefixes
df.loc[df['average_tweets_per_day'].str.startswith('INVALID_', na=False), 'average_tweets_per_day'] = \
    df['average_tweets_per_day'].str.replace('INVALID_', '', regex=False)

# convert to numeric, all non numeric values ==> nan
df['average_tweets_per_day'] = pd.to_numeric(df['average_tweets_per_day'], errors='coerce')

# if -9999 or lower, set to nan
df.loc[df['average_tweets_per_day'] <= -9999, 'average_tweets_per_day'] = np.nan

print("\n" + "="*70)
print("AFTER VALUE EXTRACTION AND COERCION:")
print("="*70)
print(f"Data type: {df['average_tweets_per_day'].dtype}")
missing_after_fix = df['average_tweets_per_day'].isnull().sum()
print(f"Total missing/invalid values (NaNs): {missing_after_fix}")

#median
print("\n" + "="*70)
print("RECALCULATING MEDIAN FOR IMPUTATION:")
print("="*70)
median_value = df['average_tweets_per_day'].median()
print(f"New Median (calculated from clean data): {median_value:.3f}")

# impute missing with median
print("\n" + "="*70)
print("IMPUTING MISSING VALUES:")
print("="*70)
print(f"Missing values before imputation: {missing_after_fix}")

df['average_tweets_per_day'] = df['average_tweets_per_day'].fillna(median_value)

print(f"Missing values after imputation: {df['average_tweets_per_day'].isna().sum()}")
print(f"✓ All missing values imputed with {median_value:.3f}")


# convert to float
df['average_tweets_per_day'] = df['average_tweets_per_day'].astype('float64')

print("\n" + "="*70)
print("FINAL CLEANING SUMMARY:")
print("="*70)
print(f"Final Data Type: {df['average_tweets_per_day'].dtype}")
print(f"Final Missing Values: {df['average_tweets_per_day'].isnull().sum()}")
print("\n✓ 'average_tweets_per_day' column successfully cleaned and converted to 'float64'")

CLEANING: average_tweets_per_day

AFTER VALUE EXTRACTION AND COERCION:
Data type: float64
Total missing/invalid values (NaNs): 24

RECALCULATING MEDIAN FOR IMPUTATION:
New Median (calculated from clean data): 1.488

IMPUTING MISSING VALUES:
Missing values before imputation: 24
Missing values after imputation: 0
✓ All missing values imputed with 1.488

FINAL CLEANING SUMMARY:
Final Data Type: float64
Final Missing Values: 0

✓ 'average_tweets_per_day' column successfully cleaned and converted to 'float64'


In [ ]:
# CLEANING: account_age_days 

print("="*70)
print("CLEANING: account_age_days")
print("="*70)

# convert to string
df['account_age_days'] = df['account_age_days'].astype(str).str.strip()

# invalid_prefix handling
df.loc[df['account_age_days'].str.startswith('INVALID_', na=False), 'account_age_days'] = \
    df['account_age_days'].str.replace('INVALID_', '', regex=False)

# convert to numeric, all non numeric values ==> nan
df['account_age_days'] = pd.to_numeric(df['account_age_days'], errors='coerce')

#if -9999 or lower, set to nan
df.loc[df['account_age_days'] <= -9999, 'account_age_days'] = np.nan

print("\n" + "="*70)
print("AFTER VALUE EXTRACTION AND COERCION:")
print("="*70)
print(f"Data type: {df['account_age_days'].dtype}")
missing_after_fix = df['account_age_days'].isnull().sum()
print(f"Total missing/invalid values (NaNs): {missing_after_fix}")

#median
print("\n" + "="*70)
print("RECALCULATING MEDIAN FOR IMPUTATION:")
print("="*70)
median_value = df['account_age_days'].median()
print(f"New Median (calculated from clean data): {median_value:.0f} days")

#median
print("\n" + "="*70)
print("IMPUTING MISSING VALUES:")
print("="*70)
print(f"Missing values before imputation: {missing_after_fix}")

df['account_age_days'] = df['account_age_days'].fillna(median_value)

print(f"Missing values after imputation: {df['account_age_days'].isna().sum()}")
print(f"✓ All missing values imputed with {median_value:.0f} days")


df['account_age_days'] = df['account_age_days'].astype('int64')

print("\n" + "="*70)
print("FINAL CLEANING SUMMARY:")
print("="*70)
print(f"Final Data Type: {df['account_age_days'].dtype}")
print(f"Final Missing Values: {df['account_age_days'].isnull().sum()}")
print("\n✓ 'account_age_days' column successfully cleaned and converted to 'int64'")

CLEANING: account_age_days

AFTER VALUE EXTRACTION AND COERCION:
Data type: float64
Total missing/invalid values (NaNs): 23

RECALCULATING MEDIAN FOR IMPUTATION:
New Median (calculated from clean data): 3219 days

IMPUTING MISSING VALUES:
Missing values before imputation: 23
Missing values after imputation: 0
✓ All missing values imputed with 3219 days

FINAL CLEANING SUMMARY:
Final Data Type: int64
Final Missing Values: 0

✓ 'account_age_days' column successfully cleaned and converted to 'int64'


In [ ]:
# List of columns to drop
columns_to_drop = ['Unnamed: 0', 'created_at']

print("="*70)
print(f"DROPPING COLUMNS: {', '.join(columns_to_drop)}")
print("="*70)
df.drop(columns=columns_to_drop, inplace=True)

print(f"✓ Columns successfully dropped.")
print(f"New DataFrame shape: {df.shape}")
print(f"Remaining columns: {list(df.columns)}")

DROPPING COLUMNS: Unnamed: 0, created_at
✓ Columns successfully dropped.
New DataFrame shape: (37446, 18)
Remaining columns: ['default_profile', 'default_profile_image', 'description', 'favourites_count', 'followers_count', 'friends_count', 'geo_enabled', 'id', 'lang', 'location', 'profile_background_image_url', 'profile_image_url', 'screen_name', 'statuses_count', 'verified', 'average_tweets_per_day', 'account_age_days', 'account_type']


In [ ]:

#screen_name investigation

print("="*70)
print("🔍 INSPECTING: screen_name")
print("="*70)

# Convert to string to handle NaNs and check basic counts
df['screen_name'] = df['screen_name'].astype(str).str.strip()

missing_count = (df['screen_name'].str.lower() == 'nan').sum()
print(f"Total missing values (NaNs): {missing_count}")
print(f"Top values (to check for placeholder names):")
print(df['screen_name'].value_counts().head(10).to_markdown())

#  Check for Non-Standard Characters and Length

allowed_chars = r'[^a-zA-Z0-9_]'
non_standard_names = df[
    (df['screen_name'].str.contains(allowed_chars, regex=True, na=False)) |  
    (df['screen_name'].str.len() < 3) |                                     # Too short (Twitter requires 4)
    (df['screen_name'].str.len() > 15)                                    # Too long (Twitter limit is 15)
]['screen_name'].value_counts()

print("\n" + "="*70)
print("NON-STANDARD NAMES (Invalid Chars, Too Short/Long):")
print("="*70)

if not non_standard_names.empty:
    print(f"Found {non_standard_names.sum()} non-standard names.")
    print(non_standard_names.head(20).to_markdown())
else:
    print("No obvious non-standard names or invalid characters found.")

# Check for obvious placeholders like 'unknown'
placeholder_names = df[df['screen_name'].str.lower().isin(['unknown', 'nan', 'test', 'user'])]['screen_name'].value_counts()
print("\nObvious Placeholder Names:")
print(placeholder_names.to_markdown())

screen_name_465 = df['screen_name'].iloc[465]

print("="*70)
print(f"🔍 SCREEN NAME (Twitter Handle) for Row Index 465:")
print("="*70)
print(f"Row Index 465, Screen Name: {screen_name_465}")

🔍 INSPECTING: screen_name
Total missing values (NaNs): 10
Top values (to check for placeholder names):
| screen_name    |   count |
|:---------------|--------:|
| nan            |      10 |
| 999999         |       2 |
| unknown        |       2 |
| JoleonLescott  |       1 |
| Ayat_140       |       1 |
| carrieanninaba |       1 |
| sherine        |       1 |
| cordensmaureen |       1 |
| GhamGraham     |       1 |
| jainabdulaziz  |       1 |

NON-STANDARD NAMES (Invalid Chars, Too Short/Long):
Found 9 non-standard names.
| screen_name             |   count |
|:------------------------|--------:|
| vc                      |       1 |
| EW                      |       1 |
| INVALID_IAMFRIMPONG26   |       1 |
| INVALID_kimchiri        |       1 |
| HP                      |       1 |
| YG                      |       1 |
| INVALID_CapsulaCinefila |       1 |
| INVALID_woahjergi       |       1 |
| INVALID_kekaneshrikant  |       1 |

Obvious Placeholder Names:
| screen_name   |   co

In [ ]:
# Define the placeholder to use for missing/invalid names
MISSING_NAME_PLACEHOLDER = 'MissingName'

print("="*70)
print("CLEANING: screen_name")
print("="*70)

# Ensure all are strings and standardized
df['screen_name'] = df['screen_name'].astype(str).str.strip()

# 'INVALID_' prefixes
print("Extracting valid names from 'INVALID_' prefixes...")
df.loc[df['screen_name'].str.startswith('INVALID_', na=False), 'screen_name'] = \
    df['screen_name'].str.replace('INVALID_', '', regex=False)

#impute with placeholder name
print("Imputing missing and generic placeholder values...")
placeholders_to_replace = ['nan', 'unknown', '999999']
df.loc[df['screen_name'].str.lower().isin(placeholders_to_replace), 'screen_name'] = MISSING_NAME_PLACEHOLDER

# replace numeric screen names with placeholder
print("Replacing purely numeric screen names with placeholder...")
mask_numeric = df['screen_name'].str.match(r'^\d+$', na=False)
df.loc[mask_numeric, 'screen_name'] = MISSING_NAME_PLACEHOLDER

print("\n" + "="*70)
print("FINAL CLEANING SUMMARY:")
print("="*70)
print(f"Final Missing Values (Placeholder Count): {(df['screen_name'] == MISSING_NAME_PLACEHOLDER).sum()}")
print(f"Remaining purely numeric screen names: {df['screen_name'].str.match(r'^\d+$', na=False).sum()}")
print(f"Sample values after cleaning: {list(df['screen_name'].head())}")
print("\n✓ 'screen_name' successfully cleaned and standardized.")


CLEANING: screen_name
Extracting valid names from 'INVALID_' prefixes...
Imputing missing and generic placeholder values...
Replacing purely numeric screen names with placeholder...

FINAL CLEANING SUMMARY:
Final Missing Values (Placeholder Count): 65
Remaining purely numeric screen names: 0
Sample values after cleaning: ['best_in_dumbest', 'CJRubinPhoto', 'SVGEGENT', 'TinkerVHELPK5', 'JoleonLescott']

✓ 'screen_name' successfully cleaned and standardized.


In [20]:


# description
# ======================================================================

print("="*70)
print("🔍 INSPECTING: description (Text Column)")
print("="*70)

# 1. Handle Missing Values
# First, count all the different ways 'missing' shows up (NaN, empty string, etc.)
# Convert to string and strip for unified comparison
df['description_str'] = df['description'].astype(str).str.strip()

# Check for NaN (original missing values)
original_nan_count = df['description'].isnull().sum()
print(f"Original Missing Values (NaN): {original_nan_count}")

# Check for common textual placeholders
print("\nTop 20 Value Counts (Case-Insensitive):")
# Use value_counts on the string column, mapping all to lowercase for better grouping
description_counts = df['description_str'].str.lower().value_counts(dropna=False)
print(description_counts.head(20).to_markdown())

# Check for placeholders like a single dot, underscore, etc.
placeholders_to_check = ['.', '-', 'none', 'unknown', 'nan', '']
print("\nSpecific Placeholder Counts:")
for p in placeholders_to_check:
    count = (df['description_str'].str.lower() == p).sum()
    if count > 0:
        print(f" - '{p}' count: {count}")

# Check for names that are simply URLs or seem like accidental data entry
url_like_descriptions = df[df['description_str'].str.lower().str.contains('http|www', na=False)].shape[0]
print(f"\nDescriptions containing URLs (These are useful signals!): {url_like_descriptions}")

🔍 INSPECTING: description (Text Column)
Original Missing Values (NaN): 7271

Top 20 Value Counts (Case-Insensitive):
| description_str                                                                       |   count |
|:--------------------------------------------------------------------------------------|--------:|
| nan                                                                                   |    7271 |
| actor                                                                                 |      17 |
| blacklivesmatter                                                                      |      16 |
| .                                                                                     |      14 |
| hi                                                                                    |      10 |
| experience is one thing you can't get for nothing.                                    |       7 |
| i am not afraid of death; i just don't want to be there when it happens.         

In [ ]:

# CLEANING: description 

print("="*70)
print("CLEANING: description")
print("="*70)

#Impute NaN  with an empty string ('')
original_nan_count = df['description'].isnull().sum()
df['description'] = df['description'].fillna('')

print(f"Original Missing Values (NaN): {original_nan_count}")
print(f"✓ {original_nan_count} NaN values imputed with an empty string ('').")

# ensure the column isstring 
df['description'] = df['description'].astype(str)

print("\n" + "="*70)
print("FINAL CLEANING SUMMARY:")
print("="*70)
print(f"Final Missing Values (NaN count): {df['description'].isnull().sum()}")
print(f"Sample values after cleaning: {list(df['description'].head().str.strip())}")
print("\n✓ 'description' column successfully cleaned for feature engineering.")

CLEANING: description
Original Missing Values (NaN): 7271
✓ 7271 NaN values imputed with an empty string ('').

FINAL CLEANING SUMMARY:
Final Missing Values (NaN count): 0
Sample values after cleaning: ['Blame xaiax, Inspired by MakingInvisible, using cmu phonetic data to produce incongruous matches.\nSome images via Lorem Flickr.', 'Photographing the American West since 1980. I specialize in location portraits & events, both indoors & outside, using natural light & portable studio lighting.', 'Scruffy looking nerf herder and twitch broadcaster\n\nContact  svgegentgmail.com', 'Wife.Godmother.Friend.Feline Fanatic! Assistant Principal, Vestavia Hills Elementary Liberty Park', 'Loan coach at mancity & Aspiring DJ']

✓ 'description' column successfully cleaned for feature engineering.


In [ ]:
# Check how many duplicates exist in the 'id' column
duplicate_count = df['id'].duplicated().sum()
print(f"Number of duplicate IDs: {duplicate_count}")

#  Display all duplicate rows 
duplicates = df[df['id'].duplicated(keep=False)]
print("\nDuplicate rows based on 'id':")
print(duplicates.head())

#drop duplicates
df_no_duplicates = df.drop_duplicates(subset='id', keep='first')

Number of duplicate IDs: 14

Duplicate rows based on 'id':
       default_profile  default_profile_image  \
465               True                  False   
466              False                  False   
5574             False                  False   
5575             False                  False   
11119            False                  False   

                                             description  favourites_count  \
465                                                                   2068   
466                                                                   2068   
5574                      EL SEOR ES MI REFUGIO ETERNO !              2068   
5575                                                                  2068   
11119  Military beat washingtonpost. UMassJournalism ...               181   

       followers_count  friends_count  geo_enabled   id lang  \
465                365            296        False  NaN  NaN   
466                365            296        Fals

In [ ]:
# drop
columns_to_drop = [
    'id',
    'lang',
    'location',
    'profile_background_image_url',
    'profile_image_url',
    'description_str'
]

existing_cols = [col for col in columns_to_drop if col in df.columns]
df = df.drop(columns=existing_cols)

print("Dropped columns:", existing_cols)
print("\nNew shape:", df.shape)
print("\nRemaining columns:")
print(df.columns.tolist())

Dropped columns: ['id', 'lang', 'location', 'profile_background_image_url', 'profile_image_url', 'description_str']

New shape: (37446, 13)

Remaining columns:
['default_profile', 'default_profile_image', 'description', 'favourites_count', 'followers_count', 'friends_count', 'geo_enabled', 'screen_name', 'statuses_count', 'verified', 'average_tweets_per_day', 'account_age_days', 'account_type']


In [ ]:

output_filename = 'bot_detection_cleaned.csv'

print("="*70)
print(f"SAVING DATASET TO: {output_filename}")
print("="*70)

df.to_csv(output_filename, index=False)

print(f"✓ Dataset successfully saved to '{output_filename}'")
print("\nYour dataset is now safely stored with all quantitative features cleaned!")

SAVING DATASET TO: bot_detection_cleaned.csv
✓ Dataset successfully saved to 'bot_detection_cleaned.csv'

Your dataset is now safely stored with all quantitative features cleaned!


In [26]:
df = pd.read_csv("bot_detection_cleaned.csv")

print("Shape:", df.shape)
print("\nColumns and types:")
print(df.dtypes)
print("\nFirst few rows:")
print(df.head())
print("\nBasic stats:")
print(df.describe())
print("\nMissing values:")
print(df.isnull().sum())

print(df['account_type'].value_counts())

Shape: (37446, 13)

Columns and types:
default_profile              bool
default_profile_image        bool
description                object
favourites_count            int64
followers_count             int64
friends_count               int64
geo_enabled                  bool
screen_name                object
statuses_count              int64
verified                     bool
average_tweets_per_day    float64
account_age_days            int64
account_type               object
dtype: object

First few rows:
   default_profile  default_profile_image  \
0            False                  False   
1            False                  False   
2            False                  False   
3             True                  False   
4            False                  False   

                                         description  favourites_count  \
0  Blame xaiax, Inspired by MakingInvisible, usin...                 4   
1  Photographing the American West since 1980. I ...               53

In [ ]:
print("="*60)
print("INITIAL STATE")
print("="*60)
print(f"Total rows: {len(df)}")
print(f"Missing account_type: {df['account_type'].isna().sum()}")
print(f"\nUnique values in account_type:")
print(df['account_type'].value_counts(dropna=False))

# Check for any weird whitespace or hidden characters
print("\nChecking for non-standard values:")
unique_vals = df['account_type'].unique()
for val in unique_vals:
    if pd.notna(val):
        print(f"'{val}' | Length: {len(str(val))} | Type: {type(val)}")

df_clean = df.dropna(subset=['account_type'])

print("="*60)
print("FINAL DATASET")
print("="*60)
print(f"Total rows: {len(df_clean)}")
print(f"Rows dropped: {len(df) - len(df_clean)}")
print(f"\nClass distribution:")
print(df_clean['account_type'].value_counts())
print(f"\nClass balance: {df_clean['account_type'].value_counts(normalize=True)}")
print(f"Missing values: {df_clean['account_type'].isna().sum()}")

# Verify
assert df_clean['account_type'].isna().sum() == 0, "Still have missing labels!"
assert df_clean['account_type'].isin(['human', 'bot']).all(), "Invalid labels found!"


INITIAL STATE
Total rows: 37446
Missing account_type: 21

Unique values in account_type:
account_type
human    25005
bot      12420
NaN         21
Name: count, dtype: int64

Checking for non-standard values:
'bot' | Length: 3 | Type: <class 'str'>
'human' | Length: 5 | Type: <class 'str'>
FINAL DATASET
Total rows: 37425
Rows dropped: 21

Class distribution:
account_type
human    25005
bot      12420
Name: count, dtype: int64

Class balance: account_type
human    0.668136
bot      0.331864
Name: proportion, dtype: float64
Missing values: 0

✓ Dataset ready for modeling


In [ ]:
print("="*60)
print("VALIDATING DATA AGAINST ACCOUNT AGE")
print("="*60)

# Calculate theoretical maximums based on account age
print("\n1. CHECKING THEORETICAL LIMITS:")
print("-"*60)

# Theoretical max tweets = account_age_days * tweets_per_day
# Twitter rate limit: ~2400 tweets/day
df_clean['theoretical_max_tweets'] = df_clean['account_age_days'] * 2400

# Check if statuses_count exceeds what's physically possible
impossible_tweets = df_clean['statuses_count'] > df_clean['theoretical_max_tweets']
print(f"Accounts with MORE tweets than physically possible (2400/day): {impossible_tweets.sum()}")

if impossible_tweets.sum() > 0:
    print("\nSample of impossible accounts:")
    print(df_clean[impossible_tweets][['screen_name', 'statuses_count', 'account_age_days', 
                                        'theoretical_max_tweets', 'average_tweets_per_day', 
                                        'account_type']].head(10))

# 2. Validate average_tweets_per_day calculation
print("\n\n2. VALIDATING AVERAGE_TWEETS_PER_DAY CALCULATION:")
print("-"*60)

# Recalculate what it SHOULD be
df_clean['calculated_tweets_per_day'] = df_clean['statuses_count'] / df_clean['account_age_days']

# Compare with existing value
df_clean['tweets_per_day_diff'] = abs(df_clean['average_tweets_per_day'] - df_clean['calculated_tweets_per_day'])

# Check for mismatches (allowing small floating point differences)
mismatches = df_clean['tweets_per_day_diff'] > 0.01
print(f"Rows where average_tweets_per_day doesn't match calculation: {mismatches.sum()}")

if mismatches.sum() > 0:
    print("\nSample of mismatches:")
    print(df_clean[mismatches][['screen_name', 'statuses_count', 'account_age_days',
                                 'average_tweets_per_day', 'calculated_tweets_per_day',
                                 'tweets_per_day_diff']].head(10))

# 3. Check the 999,999 values in context of account age
print("\n\n3. ANALYZING CAPPED VALUES (999,999):")
print("-"*60)

capped_tweets_per_day = df_clean[df_clean['average_tweets_per_day'] == 999999]
print(f"Accounts with tweets_per_day = 999,999: {len(capped_tweets_per_day)}")

if len(capped_tweets_per_day) > 0:
    print("\nThese accounts in detail:")
    print(capped_tweets_per_day[['screen_name', 'statuses_count', 'account_age_days',
                                  'average_tweets_per_day', 'calculated_tweets_per_day',
                                  'account_type']].to_string())
    
    # What SHOULD their tweets_per_day be?
    print("\n\nRecalculated tweets_per_day for capped accounts:")
    for idx, row in capped_tweets_per_day.iterrows():
        actual_rate = row['statuses_count'] / row['account_age_days']
        print(f"  {row['screen_name']}: Should be {actual_rate:.2f} tweets/day (not 999,999)")

# 4. Check favourites_count against activity
print("\n\n4. VALIDATING FAVOURITES_COUNT:")
print("-"*60)

# You can't favorite more tweets than you could possibly see
# maybe 10,000 tweets/day max to see/favorite
df_clean['theoretical_max_favorites'] = df_clean['account_age_days'] * 10000

impossible_favs = df_clean['favourites_count'] > df_clean['theoretical_max_favorites']
print(f"Accounts with MORE favorites than theoretically possible: {impossible_favs.sum()}")

if impossible_favs.sum() > 0:
    print("\nSample of impossible favorite counts:")
    print(df_clean[impossible_favs][['screen_name', 'favourites_count', 'account_age_days',
                                      'theoretical_max_favorites', 'account_type']].head(10))

# 5. More realistic checks for human behavior
print("\n\n5. REALISTIC HUMAN LIMITS:")
print("-"*60)

# Humans rarely tweet more than 50/day consistently, favorites >100/day is very high
df_clean['realistic_max_tweets'] = df_clean['account_age_days'] * 50
df_clean['realistic_max_favorites'] = df_clean['account_age_days'] * 100

suspicious_tweet_rate = df_clean['statuses_count'] > df_clean['realistic_max_tweets']
suspicious_fav_rate = df_clean['favourites_count'] > df_clean['realistic_max_favorites']

print(f"Accounts exceeding realistic human tweet rate (>50/day avg): {suspicious_tweet_rate.sum()}")
print(f"  Bots: {(suspicious_tweet_rate & (df_clean['account_type'] == 'bot')).sum()}")
print(f"  Humans: {(suspicious_tweet_rate & (df_clean['account_type'] == 'human')).sum()}")

print(f"\nAccounts exceeding realistic favorite rate (>100/day avg): {suspicious_fav_rate.sum()}")
print(f"  Bots: {(suspicious_fav_rate & (df_clean['account_type'] == 'bot')).sum()}")
print(f"  Humans: {(suspicious_fav_rate & (df_clean['account_type'] == 'human')).sum()}")

# 6. Summary of data quality issues
print("\n\n" + "="*60)
print("DATA QUALITY SUMMARY")
print("="*60)

issues = pd.DataFrame({
    'Issue': [
        'Physically impossible tweet count',
        'Tweets_per_day mismatch',
        'Capped tweets_per_day (999,999)',
        'Impossible favorites',
        'Suspicious tweet rate (>50/day)',
        'Suspicious favorite rate (>100/day)'
    ],
    'Count': [
        impossible_tweets.sum(),
        mismatches.sum(),
        (df_clean['average_tweets_per_day'] == 999999).sum(),
        impossible_favs.sum(),
        suspicious_tweet_rate.sum(),
        suspicious_fav_rate.sum()
    ],
    'Percentage': [
        f"{impossible_tweets.sum()/len(df_clean)*100:.2f}%",
        f"{mismatches.sum()/len(df_clean)*100:.2f}%",
        f"{(df_clean['average_tweets_per_day'] == 999999).sum()/len(df_clean)*100:.2f}%",
        f"{impossible_favs.sum()/len(df_clean)*100:.2f}%",
        f"{suspicious_tweet_rate.sum()/len(df_clean)*100:.2f}%",
        f"{suspicious_fav_rate.sum()/len(df_clean)*100:.2f}%"
    ]
})

print(issues.to_string(index=False))

# Clean up temporary columns
df_clean = df_clean.drop(['theoretical_max_tweets', 'theoretical_max_favorites',
                          'realistic_max_tweets', 'realistic_max_favorites',
                          'calculated_tweets_per_day', 'tweets_per_day_diff'], 
                         axis=1, errors='ignore')

VALIDATING DATA AGAINST ACCOUNT AGE

1. CHECKING THEORETICAL LIMITS:
------------------------------------------------------------
Accounts with MORE tweets than physically possible (2400/day): 0


2. VALIDATING AVERAGE_TWEETS_PER_DAY CALCULATION:
------------------------------------------------------------
Rows where average_tweets_per_day doesn't match calculation: 47

Sample of mismatches:
        screen_name  statuses_count  account_age_days  average_tweets_per_day  \
233     kweenklarke            4209              2760                  30.890   
262       JeniBohls            1884              3126                   1.488   
349     TomFletcher            4209              4154                   9.787   
1497      mpolletta           34502              3219                   8.191   
1714     BudMiller9            4209              2074                   0.765   
2112  erikmackenzie           38174              6402                  18.658   
2845        aRolasx           11844   

C:\Users\Badr\AppData\Local\Temp\ipykernel_15956\4279845164.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['theoretical_max_tweets'] = df_clean['account_age_days'] * 2400
C:\Users\Badr\AppData\Local\Temp\ipykernel_15956\4279845164.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['calculated_tweets_per_day'] = df_clean['statuses_count'] / df_clean['account_age_days']
C:\Users\Badr\AppData\Local\Temp\ipykernel_15956\4279845164.py:31: SettingWithCopyWarning: 
A value is trying to be

In [32]:


print("="*60)
print("FIXING DATA QUALITY ISSUES")
print("="*60)

# 1. FIX: Recalculate average_tweets_per_day for all rows
print("\n1. Recalculating average_tweets_per_day...")
df_clean['average_tweets_per_day'] = df_clean['statuses_count'] / df_clean['account_age_days']

# Verify the fix
mismatches_after = (df_clean['average_tweets_per_day'] > 1000).sum()
print(f"   ✓ Fixed calculation")
print(f"   Rows with tweets_per_day > 1000 (should be 0): {mismatches_after}")

# 2. DECISION: What to do with high-activity accounts?
print("\n\n2. Analyzing high-activity accounts...")
print("-"*60)

# Let's see the distribution of tweet rates by account type
high_tweet_rate = df_clean[df_clean['average_tweets_per_day'] > 50].copy()
print(f"\nAccounts with >50 tweets/day: {len(high_tweet_rate)}")
print(f"  Bots: {(high_tweet_rate['account_type'] == 'bot').sum()}")
print(f"  Humans: {(high_tweet_rate['account_type'] == 'human').sum()}")

print("\nTweet rate distribution for high-activity accounts:")
print(high_tweet_rate['average_tweets_per_day'].describe())

# Sample some to see if they look legitimate
print("\nSample of high-activity accounts:")
sample = high_tweet_rate.nlargest(10, 'average_tweets_per_day')[
    ['screen_name', 'average_tweets_per_day', 'statuses_count', 
     'account_age_days', 'followers_count', 'account_type']
]
print(sample.to_string())

# 3. RECOMMENDATION: Use IQR now that data is validated
print("\n\n" + "="*60)
print("RECOMMENDATION: APPLY IQR METHOD")
print("="*60)

print("""
Now that we've validated the data is physically possible:
1. ✓ Fixed the calculation errors (47 rows)
2. ✓ Fixed the 999,999 cap (1 row)
3. High activity accounts (>50/day) are legitimate but may be outliers

Next step: Apply IQR method to handle statistical outliers
- These high-activity accounts will naturally be flagged as outliers
- IQR will handle them systematically across ALL numeric features
""")

# Save the corrected data
print("\n4. Saving corrected average_tweets_per_day...")
# df_clean.to_csv('data_with_corrected_tweets_per_day.csv', index=False)

print("\n✓ Data validation and correction complete!")
print(f"Total rows: {len(df_clean)}")
print("\nReady to apply IQR method for outlier detection.")

# Quick preview of corrected stats
print("\n" + "="*60)
print("CORRECTED STATISTICS")
print("="*60)
print(df_clean[['favourites_count', 'followers_count', 'friends_count',
                'statuses_count', 'average_tweets_per_day']].describe())

FIXING DATA QUALITY ISSUES

1. Recalculating average_tweets_per_day...
   ✓ Fixed calculation
   Rows with tweets_per_day > 1000 (should be 0): 4


2. Analyzing high-activity accounts...
------------------------------------------------------------

Accounts with >50 tweets/day: 792
  Bots: 387
  Humans: 405

Tweet rate distribution for high-activity accounts:
count     792.000000
mean      110.474686
std       121.682492
min        50.016619
25%        59.621075
50%        76.006541
75%       111.286427
max      1269.054924
Name: average_tweets_per_day, dtype: float64

Sample of high-activity accounts:
           screen_name  average_tweets_per_day  statuses_count  account_age_days  followers_count account_type
37234       10printbot             1269.054924         1340122              1056               15          bot
19527         sectest9             1191.285891         1925118              1616            32444          bot
8150   IndieGameDevBot             1080.593721         23

In [ ]:
# Continue with IQR method on corrected data
print("\n\n" + "="*60)
print("APPLYING IQR METHOD ON CORRECTED DATA")
print("="*60)

numeric_cols = ['favourites_count', 'followers_count', 'friends_count', 
                'statuses_count', 'average_tweets_per_day']

def apply_iqr_capping(df, columns, multiplier=1.5):
    """Cap outliers using IQR method"""
    df_capped = df.copy()
    
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = max(0, Q1 - multiplier * IQR)
        upper_bound = Q3 + multiplier * IQR
        
        # Count outliers before capping
        outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
        
        # Cap values
        df_capped[col] = df_capped[col].clip(lower=lower_bound, upper=upper_bound)
        
        print(f"{col}:")
        print(f"  Upper bound: {upper_bound:,.2f}")
        print(f"  Outliers capped: {outliers} ({outliers/len(df)*100:.2f}%)")
    
    return df_capped


df_final = apply_iqr_capping(df_clean, numeric_cols, multiplier=1.5)

print("\n" + "="*60)
print("FINAL CLEANED DATASET")
print("="*60)
print(f"Total rows: {len(df_final)}")
print(f"\nClass distribution:")
print(df_final['account_type'].value_counts())
print(f"\nFinal statistics:")
print(df_final[numeric_cols].describe())

# Save final cleaned data
df_final.to_csv('final_cleaned_twitter_bot_data.csv', index=False)




APPLYING IQR METHOD ON CORRECTED DATA
favourites_count:
  Upper bound: 21,654.50
  Outliers capped: 5193 (13.88%)
followers_count:
  Upper bound: 21,010.00
  Outliers capped: 8202 (21.92%)
friends_count:
  Upper bound: 2,174.50
  Outliers capped: 4102 (10.96%)
statuses_count:
  Upper bound: 40,853.00
  Outliers capped: 4507 (12.04%)
average_tweets_per_day:
  Upper bound: 13.07
  Outliers capped: 4527 (12.10%)

FINAL CLEANED DATASET
Total rows: 37425

Class distribution:
account_type
human    25005
bot      12420
Name: count, dtype: int64

Final statistics:
       favourites_count  followers_count  friends_count  statuses_count  \
count      37425.000000     37425.000000   37425.000000    37425.000000   
mean        5923.321456      5521.356232     604.899158    11442.270701   
std         7694.700205      8604.946855     723.230814    13935.328725   
min            0.000000         0.000000       0.000000        0.000000   
25%          362.000000        35.000000      37.000000     

In [35]:
df_please_work = pd.read_csv("final_cleaned_twitter_bot_data.csv")

print("Shape:", df_please_work.shape)
print("\nColumns and types:")
print(df_please_work.dtypes)
print("\nFirst few rows:")
print(df_please_work.head())
print("\nBasic stats:")
print(df_please_work.describe())
print("\nMissing values:")
print(df_please_work.isnull().sum())

print(df_please_work['account_type'].value_counts())

Shape: (37425, 13)

Columns and types:
default_profile              bool
default_profile_image        bool
description                object
favourites_count          float64
followers_count             int64
friends_count             float64
geo_enabled                  bool
screen_name                object
statuses_count              int64
verified                     bool
average_tweets_per_day    float64
account_age_days            int64
account_type               object
dtype: object

First few rows:
   default_profile  default_profile_image  \
0            False                  False   
1            False                  False   
2            False                  False   
3             True                  False   
4            False                  False   

                                         description  favourites_count  \
0  Blame xaiax, Inspired by MakingInvisible, usin...               4.0   
1  Photographing the American West since 1980. I ...             536.